# OpenStreetMap Data Case Study

## Map Area

Location Used: Murfreesboro, TN

https://www.openstreetmap.org/relation/197137 

I chose the map of Murfreesboro because it is the city where I live. It is in Tennessee and could be considered a suburb of Nashville. It is the geographic center of Tennessee and was formerly the state capital. It is currently one of the fastest growing cities in the country. 

## Problems Encountered in the Map

By pulling out all the street types (for example street, way, road) in the data set I see that there are a few that have multiple spellings and abbreviations. Boulevard is spelled out but also abbreviated as Blvd. The same is true for court, drive, parkway, road, and street. There also appears to be a zip code (postal code) entered as a street type. 

I also pulled out the zipcodes in the OSM data. These are tagged as postcode in the file. I found that there were a couple that were not formatted correctly. They were longer than 5 digits. Also, there were fourteen different zipcodes in the file. According to the United States Postal Service website there are only seven zipcodes in Murfreesboro, TN. (https://tools.usps.com/zip-code-lookup.htm?bycitystate)

In [2]:
# importing the modules that will be needed
import csv 
import requests 
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import pprint
import re
import os

In [2]:
#The different street types and how often they appear in the data
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("map.xml", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()


#112: 2
37128: 1
Avenue: 3
B220: 1
Blvd: 1
Boulevard: 84
Circle: 1
Court: 73
Crossing: 1
Ct: 1
Dr: 1
Drive: 189
East: 2
Fork: 1
Highway: 12
Lane: 144
Parkway: 30
Pass: 2
Pike: 26
Pkwy: 2
Place: 16
Rd: 7
Road: 69
Row: 1
Square: 1
St: 1
Street: 38
Terrace: 2
Trace: 1
Trail: 1
Way: 5
West: 1


In [5]:
#Postalcodes/Zipcodes audit
mboro_pcodes = 'map.xml'

#sorting the properly and improperly formatted zipcodes into their own lists
def audit_postal_code(error_codes, postal_codes, zipcode): 
    if zipcode.isdigit() == False:
        error_codes.append(zipcode)
    elif len(zipcode) != 5:
        error_codes.append(zipcode)
    else:
        postal_codes.update([zipcode])

#defining where the zipcodes can be pulled from in the xml data
#they are called postcodes in the OSM file, not zipcodes
def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode" or elem.attrib['k'] == "postal_code")

#opening the OSM file and pulling the zipcodes out to sort them into an error list and correct list
def audit_post(osmfile):
    osm_file = open(osmfile, "r")
    error_codes = []
    postal_codes = set([])
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(error_codes, postal_codes, tag.attrib["v"]) 
    return error_codes, postal_codes

error_list, postcode_list = audit_post(mboro_pcodes)

# List of Murfreesboro, TN zip codes
murfreesboro_postalcodes = [ '37127', '37128', '37129', '37130', '37131', 
                        '37132', '37133']

# number of zip codes in the osm file that are outside of Murfreesboro, TN
count = 0
for i in postcode_list:
    if i not in murfreesboro_postalcodes:
        count += 1

print "These are the zipcodes in the Murfreesboro, TN OSM file:", postcode_list
print count, "of these zipcodes are outside of Murfreesboro, TN"
print "These are the zipcodes that did not meet the standard format", error_list

These are the zipcodes in the Murfreesboro, TN OSM file: set(['37037', '37027', '37167', '37013', '37014', '37086', '37130', '37128', '37129', '37135', '37067', '37127', '37153', '37211'])
10 of these zipcodes are outside of Murfreesboro, TN
These are the zipcodes that did not meet the standard format ['37132-0001', '37132-0001']


## Cleaning and Shaping the Data

This section shows the code/processes for cleaning the data and then shaping it to be inserted into a SQL database. 

For the addresses I am updating the abbreviations of the street types to full words. I did find a zip code and what appeared to be an apartment number in the street type data, but I did not clean this data. For the zip codes I am updating the ones that did not meet the standard 5 digit format. There were also several zipcodes that were outside the Murfreesboro area according to the USPS website that I did not clean. The additonal address and zipcode issues should be addressed and cleaned, but this would involve the type of research and cost that is beyond the scope of this project. A suggestion for cleaning this data would include using the USPS services for cleaning address lists (https://pe.usps.com/BusinessMail101?ViewName=CheckTheAddresses). 

In [6]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import schema

#From OSM of Murfreesboro, TN, changed to XML
MAP_FILE = "map.xml"

#Regular expression to help in the search through the XML file for the street types (such as drive, dr, ave...)  
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# These are the correct street types, this list is used for checking the street types 
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Road", 
            "Parkway", "Plaza", "Broadway", "Circle", "Driveway", "Highway", "Park", "Lane"]

# This dictionary is used to correct the street types from their 
#incorrect entry in the map.xml file to the correct type
mapping = {"St": "Street",
           "st": "Street",
           "Street.": "Street", 
           "street": "Street", 
           "ST": "Street",
           "Blvd":"Boulevard",
           "Ave": "Avenue", 
           "Pkwy": "Parkway", 
           "Ct": "Court",  
           "Dr": "Drive", 
           "Rd": "Road", 
           "Hwy": "Highway"}


# Make sure the fields order in the csvs matches the column order in the
# sql table schema
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

#looks for the incorrect street types in the street names by comparing them to the "expected" list
#and then puts them in a list called street_types
#uses the regular expression "street_type_re" defined prevously to locate the street type within the street name
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
#finds the street names in the map.xml file
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

#executes the audit_street_type and is_street_name functions to fill the street_types dictionary
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

#fixes the street type in the street name
def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        for i in mapping:
            if i == m.group():
                name = re.sub(street_type_re, mapping[i], name)
    return name

#finds the zip codes in the address 
def is_postcode(elem): 
    return (elem.attrib['k'] == "addr:postcode" or elem.attrib['k'] == "postal_code")

#creates a list of zipcodes
def audit_postcode(postcodes, postcode):
    postcodes[postcode].add(postcode)
    return postcodes

#updates/cleans the zipcodes 
def update_postcode(postcode):
    if re.findall(r'^\d{5}$', postcode): # 5 digits
        valid_postcode = postcode
        return valid_postcode  
    else:
        return None

#Shape each element into several data structures
#Clean and shape node or way XML element to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    
    node_attribs = {} 
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):  
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                node_tag = {} 
                node_tag['id'] = element.attrib['id'] 
                node_tag['value'] = tag.attrib['v']  

                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    node_tag['type'] = 'regular'
                    node_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+):', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])
                    node_tag['type'] = bef_colon[0]
                    node_tag['key'] = aft_colon[0]
                    if node_tag['type'] == "addr" and node_tag['key'] == "street":
                        # update street name
                        node_tag['value'] = update_name(tag.attrib['v'], mapping) 
                    elif node_tag['type'] == "addr" and node_tag['key'] == "postcode":
                        # update post code
                        node_tag['value'] = update_postcode(tag.attrib['v']) 
            tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for i in WAY_FIELDS:
            way_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                way_tag = {}
                way_tag['id'] = element.attrib['id'] 
                way_tag['value'] = tag.attrib['v']
                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+?):+[a-z]', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])

                    way_tag['type'] = bef_colon[0]
                    way_tag['key'] = aft_colon[0]
                    if way_tag['type'] == "addr" and way_tag['key'] == "street":
                        way_tag['value'] = update_name(tag.attrib['v'], mapping) 
                    elif way_tag['type'] == "addr" and way_tag['key'] == "postcode":
                        way_tag['value'] = update_postcode(tag.attrib['v']) 
            tags.append(way_tag)
        position = 0
        for tag in element.iter("nd"):  
            nd = {}
            nd['id'] = element.attrib['id'] 
            nd['node_id'] = tag.attrib['ref'] 
            nd['position'] = position  
            position += 1
            
            way_nodes.append(nd)
    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
#Yield element if it is the right type of tag
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

#Raise ValidationError if element does not match schema
def validate_element(element, validator, schema=SCHEMA):
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

#Extend csv.DictWriter to handle Unicode input
class UnicodeDictWriter(csv.DictWriter, object):
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
#Iteratively process each XML element and write to csv(s)
def process_map(file_in, validate):

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()



        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(MAP_FILE, validate=False)

## Overview of the Data

This code is for establishing a connection to the database I created in MySQL using the OSM data. I used the csv files created in the process above to create the database. To be able to run and show my SQL queries I had to include this to create the connection between the MySQL database and my Jupyter Notebook. 

In [3]:
#connect to MySQL
import mysql.connector
#db = mysql.connector.connect(MySQL_DataWrangle_OSM)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="****",
  database="default_schema",
  use_pure=True
)

#confirm the connection
print(mydb)



This is the code and results that show the size of each csv file that was imported into the database. They were created during the cleaning and shaping process. The size is shown in bytes.

In [6]:
#links to the csv files
nodes_csv = 'nodes.csv'
ways_csv = 'ways.csv'
nodestags_csv = 'nodes_tags.csv'
waystags_csv = 'ways_tags.csv'
waysnodes_csv = 'ways_nodes.csv'

# Get the size (in bytes) of specified path  
size_nodes = os.path.getsize(nodes_csv) 
size_ways = os.path.getsize(ways_csv) 
size_nodestags = os.path.getsize(nodestags_csv) 
size_waystags = os.path.getsize(waystags_csv) 
size_waysnodes = os.path.getsize(waysnodes_csv) 
  
# Print the size (in bytes) of specified path  
print("Size (In bytes) of '%s':" %nodes_csv, size_nodes)
print("Size (In bytes) of '%s':" %ways_csv, size_ways)
print("Size (In bytes) of '%s':" %nodestags_csv, size_nodestags)
print("Size (In bytes) of '%s':" %waystags_csv, size_waystags)
print("Size (In bytes) of '%s':" %waysnodes_csv, size_waysnodes)

("Size (In bytes) of 'nodes.csv':", 40649303)
("Size (In bytes) of 'ways.csv':", 3137016)
("Size (In bytes) of 'nodes_tags.csv':", 784549)
("Size (In bytes) of 'ways_tags.csv':", 4831559)
("Size (In bytes) of 'ways_nodes.csv':", 13339926)


There are 687 unique users in the nodes and ways files for the Murfreesboro, TN OSM data.

In [31]:
#code for number of unique users
mycursor = mydb.cursor()
sql = "SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print('The number of unique users in the nodes and ways files:', myresult)

('The number of unique users in the nodes and ways files:', [(687,)])


There are 67,085 nodes and 51,792 ways in my OSM data. 

In [26]:
#code to show the number of nodes
mycursor = mydb.cursor()

sql = "SELECT COUNT(*) FROM nodes"
mycursor.execute(sql)

myresult = mycursor.fetchall()

for x in myresult:
  print('The number of nodes is:', x)

('The number of nodes:', (67085,))


In [58]:
#code to show the number of ways
mycursor = mydb.cursor()

sql = "SELECT COUNT(*) FROM ways"
mycursor.execute(sql)

myresult = mycursor.fetchall()

for x in myresult:
  print('The number of ways is:', x)

('The number of ways is:', (51792,))


Since moving to Murfreesboro several years ago I have noticed that there are always small, personal planes buzzing around. Also, Middle Tennessee State University, a large college located in Murfreesboro, has one of the largest collegiate aviation programs in the country. So I was curious about what the OSM data would say regarding airports and landing strips in the area. I used two queries to count how many aerodomes were in the nodes_tags and ways_tags files. According to the OSM Wiki an aeroway "describes the fixed physical infrastructure associated with air travel and space flight, including airports, runways, helipads, and terminal buildings" and an aerodome includes airports, aerodromes, airfields, and landing strips. The queries returned 15 instances. This did not surprise me due to the enthusiasm for flying I have observed in the area. 

OSM Wiki: https://wiki.openstreetmap.org/wiki/Aeroways

MTSU Aerospace program: https://www.mtsu.edu/aerospace/

In [76]:
#getting the aeroways - aerodromes from the nodes_tags table
mycursor = mydb.cursor()
sql = "SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='aerodrome') i ON nodes_tags.id=i.id WHERE nodes_tags.key='aeroway' GROUP BY nodes_tags.value ORDER BY num DESC"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print (myresult)

[(u'aerodrome', 13)]


In [3]:
#getting the aeroways - aerodromes from the ways_tags table
mycursor = mydb.cursor()
sql = "SELECT ways_tags.value, COUNT(*) as num FROM ways_tags JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value='aerodrome') i ON ways_tags.id=i.id WHERE ways_tags.key='aeroway' GROUP BY ways_tags.value ORDER BY num DESC"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print (myresult)

[(u'aerodrome', 2)]


My family has an active, two year old labrador retriever. So I was curious if there were any entries in the OSM data for my area related to dogs. First I ran a query to find any dog parks. These can be found in the nodes_tags and ways_tags tables, in the key column under leasure and in the value column under dag_parks. My query found 3 dog parks. Then I wanted to see if there were any pet stores. First I queried the same two tables using the key column looking for the different types if shops. The query included pet as a shop. Then I queried the tables for any instance of pet and found two pet shops. 

On a national level the ownership of dogs and spending on pets is increasing. There also seems to be an change in the way people consider dogs. More people are starting to think of them as a family member and less like "just a pet/dog". A good axample of this change in thinking is the new law making animal cruelty a federal crime. I was suprised there weren't more dog parks and pet stores in Murfreesboro. I would guess that it's not as much as people not changing thier opinions on dogs, but more about the size of Murfreesboro. It is a fast growing city but it still only has a population of around 141,000 people. 


https://www.americanpetproducts.org/press_industrytrends.asp

https://www.americanpetproducts.org/press_releasedetail.asp?id=191

https://www.npr.org/2019/11/25/782842651/trump-signs-law-making-cruelty-to-animals-a-federal-crime

https://www.census.gov/quickfacts/fact/table/murfreesborocitytennessee/PST045218

In [6]:
#code to return dog_parks from ways_tags and node_tages tables
mycursor = mydb.cursor()
sql = "SELECT value FROM ways_tags WHERE value='dog_park' UNION ALL SELECT value FROM nodes_tags WHERE value='dog_park'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print (myresult)

[(u'dog_park',), (u'dog_park',), (u'dog_park',)]


In [24]:
#code to list the types of shops in the tables
mycursor = mydb.cursor()
sql = "SELECT ways_tags.value FROM ways_tags WHERE ways_tags.key='shop' UNION SELECT nodes_tags.value FROM nodes_tags WHERE nodes_tags.key='shop'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print (myresult)

[(u'supermarket',), (u'chemist',), (u'mall',), (u'doityourself',), (u'craft',), (u'mobile_phone',), (u'convenience',), (u'jewelry',), (u'shoes',), (u'car_repair',), (u'wholesale',), (u'department_store',), (u'pet',), (u'tyres',), (u'car',), (u'ticket',), (u'yes',), (u'houseware',), (u'country_store',), (u'alcohol',), (u'storage_rental',), (u'variety_store',), (u'car_parts',), (u'fabric',), (u'pastry',), (u'antiques',), (u'motorcycle',), (u'hardware',), (u'shipping',), (u'hairdresser',), (u'nutrition_supplements',), (u'*',), (u'gift',), (u'furniture',), (u'beauty',), (u'stationery',), (u'clothes',), (u'toys',), (u'laundry',)]


In [26]:
#code to show each instance of pet in the ways_tag and nodes-tags tables
mycursor = mydb.cursor()
sql = "SELECT ways_tags.value FROM ways_tags WHERE ways_tags.value='pet' UNION ALL SELECT nodes_tags.value FROM nodes_tags WHERE nodes_tags.value='pet'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print (myresult)

[(u'pet',), (u'pet',)]


In [45]:
#close the db connection
mydb.close()

## Conclusion

The data appears to be fairly clean but I do wonder about how current it is. Murfreesboro has experienced explosive growth over the past few years which has created a boom in construction in both housing and commercial projects (see the two links below). This also means many new roads, schools and other community projects. This is a lot of change in a short amount of time. Since OSM is a community project that has many collaborators at any given time, there is no guarantee as to when and how often the data for a specific area is updated. A suggestion to help with this issue would be to recruit or advertise to people who are interested in data analytics, data analysis, coding, cartography or geodata.

https://www.dnj.com/story/news/local/2019/02/22/why-murfreesboro-growing-so-fast-tennessee-capital-name-ask-google/2927769002/

https://www.wkrn.com/news/murfreesboros-growth-attracts-big-companies-fuels-area-economy/